# Notebook 01: Pipeline d'Ingestion et d'Embedding des Données ArXiv

Ce notebook démontre le processus complet d'acquisition, de traitement, d'embedding et de stockage des articles scientifiques d'ArXiv dans notre base de données MongoDB. Il est conçu pour être une démonstration pratique des modules développés dans `src/`.

**Prérequis :**
*   Assurez-vous d'avoir exécuté le notebook `00_setup_environment.ipynb` et que votre environnement est correctement configuré (variables d'environnement chargées, clés API valides, MongoDB accessible).
*   Les bibliothèques nécessaires doivent être installées via `environment.yml` (pour Conda) ou `requirements.txt`.
*   Votre serveur Ollama (si utilisé comme fournisseur d'embedding) doit être en cours d'exécution avec le modèle d'embedding spécifié (`nomic-embed-text` par défaut) disponible.

**Étapes de ce Notebook :**
1.  **Configuration Initiale** : Imports, configuration du logging, définition des chemins et paramètres spécifiques au notebook.
2.  **Téléchargement d'Articles depuis ArXiv** : Recherche et téléchargement des PDFs et de leurs métadonnées.
3.  **Parsing des Documents** : Extraction du texte des PDFs et chargement des métadonnées.
4.  **Prétraitement du Texte** : Nettoyage (minimal ici) et segmentation (chunking) du contenu textuel.
5.  **Génération des Embeddings** : Conversion des chunks de texte en vecteurs numériques.
6.  **Stockage dans MongoDB** : Insertion des chunks et de leurs embeddings dans une collection MongoDB et création des index de recherche.

In [1]:
# --- Imports Standards et de Configuration ---
import logging
from typing import List, Optional, Dict, Any
from pathlib import Path
import re # Pour la fonction de nettoyage des noms de répertoires
import shutil # Pour le nettoyage optionnel des répertoires
import sys

# --- Configuration du Projet et Logging ---
# Assurer que la racine du projet est dans sys.path pour les imports de src/ et config/
project_root_path = Path.cwd().parent 
if str(project_root_path) not in sys.path:
    sys.path.append(str(project_root_path))
    print(f"Ajout de {project_root_path} à sys.path")

from pymongo.errors import ConnectionFailure 
from config.settings import settings
from config.logging_config import setup_logging

# --- Imports des Modules de Traitement de Données de src/ ---
from src.data_processing.arxiv_downloader import search_arxiv_papers, download_papers as execute_arxiv_downloads
from src.data_processing.document_parser import parse_document_collection, ParsedDocument # ParsedDocument importé pour type hinting si besoin
from src.data_processing.preprocessor import preprocess_parsed_documents # ParsedDocument est utilisé en interne par cette fonction
from src.data_processing.embedder import generate_embeddings_for_chunks 
from src.vector_store.mongodb_manager import MongoDBManager

# --- Configuration du Logging pour ce Notebook ---
setup_logging(level="INFO") # Changer à "DEBUG" pour des logs plus détaillés
logger = logging.getLogger("nb_01_ingestion_embedding")

logger.info("--- Initialisation du Notebook 01: Ingestion et Embedding ---")

# --- Paramètres Spécifiques à l'Exécution de ce Notebook ---
# Requête ArXiv pour ce notebook
ARXIV_QUERY_NOTEBOOK = "What are the latest advancements in using large language models for robot task planning?" 
# Nombre maximum de résultats à télécharger pour ce notebook (garder bas pour des tests rapides)
MAX_RESULTS_NOTEBOOK = 3 
# Nom de la collection MongoDB pour les chunks de ce notebook (pour isoler les données de test)
COLLECTION_NAME_NOTEBOOK = f"arxiv_chunks_nb_test_{settings.DEFAULT_EMBEDDING_PROVIDER}" # Inclure le provider dans le nom
# Noms des index pour cette collection de test
VECTOR_INDEX_NAME_NOTEBOOK = "vector_index_nb_test"
TEXT_INDEX_NAME_NOTEBOOK = "text_index_nb_test"


# --- Définition et Création des Chemins de Sortie pour ce Notebook ---
def sanitize_for_dir_name(query: str) -> str:
    """Convertit une chaîne de requête en un nom de répertoire valide."""
    s = query.lower()
    s = re.sub(r'[\s\W-]+', '_', s) # Remplace espaces et caractères non alphanumériques (sauf _) par _
    s = s.strip('_') # Enlève les _ en début/fin
    return s[:50] # Limite la longueur

# Crée un sous-répertoire unique pour les données de ce notebook basé sur la requête
corpus_sub_dir_name_nb = sanitize_for_dir_name(ARXIV_QUERY_NOTEBOOK) 
notebook_corpus_base_path = Path(settings.DATA_DIR) / "corpus" / corpus_sub_dir_name_nb

# Chemins spécifiques pour les PDFs et métadonnées téléchargés par ce notebook
NB_PDF_OUTPUT_DIR = notebook_corpus_base_path / "pdfs"
NB_METADATA_OUTPUT_DIR = notebook_corpus_base_path / "metadata"

# Optionnel: Nettoyer les répertoires de sortie avant chaque exécution
# Changez `clean_output_dirs` à True pour activer le nettoyage.
CLEAN_OUTPUT_DIRS_ON_RUN = False 
if CLEAN_OUTPUT_DIRS_ON_RUN:
    logger.info("Nettoyage des répertoires de sortie du notebook...")
    if NB_PDF_OUTPUT_DIR.exists():
        shutil.rmtree(NB_PDF_OUTPUT_DIR)
        logger.info(f"Répertoire PDF supprimé : {NB_PDF_OUTPUT_DIR}")
    if NB_METADATA_OUTPUT_DIR.exists():
        shutil.rmtree(NB_METADATA_OUTPUT_DIR)
        logger.info(f"Répertoire Metadata supprimé : {NB_METADATA_OUTPUT_DIR}")

# S'assurer que les répertoires de sortie existent
NB_PDF_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
NB_METADATA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logger.info(f"Répertoire DATA_DIR global (settings.py): {settings.DATA_DIR}")
logger.info(f"PDFs pour ce notebook seront stockés dans : {NB_PDF_OUTPUT_DIR}")
logger.info(f"Métadonnées pour ce notebook seront stockées dans : {NB_METADATA_OUTPUT_DIR}")

# --- Vérification et Affichage de la Configuration d'Embedding Active ---
logger.info(f"--- Configuration d'Embedding Active (depuis settings.py & .env) ---")
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'embedding par défaut configuré : {active_embedding_provider}")

if active_embedding_provider == "openai":
    logger.info(f"  Modèle OpenAI Embedding : {settings.OPENAI_EMBEDDING_MODEL_NAME} (Dimension: {settings.OPENAI_EMBEDDING_DIMENSION})")
    if not settings.OPENAI_API_KEY:
        logger.error("  ERREUR : OPENAI_API_KEY n'est pas configurée dans .env. L'embedding OpenAI échouera.")
elif active_embedding_provider == "huggingface":
    # Note: Pour HuggingFace, on suppose l'utilisation de sentence-transformers via langchain, pas d'appel API direct ici.
    logger.info(f"  Modèle HuggingFace Embedding : {settings.HUGGINGFACE_EMBEDDING_MODEL_NAME} (Dimension: {settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION})")
elif active_embedding_provider == "ollama":
    logger.info(f"  Modèle Ollama Embedding : {settings.OLLAMA_EMBEDDING_MODEL_NAME} (Dimension: {settings.OLLAMA_EMBEDDING_MODEL_DIMENSION})")
    logger.info(f"  URL de base Ollama : {settings.OLLAMA_BASE_URL}")
    if not settings.OLLAMA_BASE_URL:
        logger.error("  ERREUR : OLLAMA_BASE_URL n'est pas configurée. L'embedding Ollama échouera.")
    logger.info(f"  ASSUREZ-VOUS que le modèle '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est disponible sur votre serveur Ollama (ex: via 'ollama pull {settings.OLLAMA_EMBEDDING_MODEL_NAME}').")
else:
    logger.error(f"ERREUR : Fournisseur d'embedding inconnu ou non supporté dans ce notebook ('{active_embedding_provider}') configuré dans settings.py. La génération d'embeddings échouera.")

logger.info("Configuration initiale du notebook terminée et paramètres définis.")

2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - --- Initialisation du Notebook 01: Ingestion et Embedding ---
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Répertoire DATA_DIR global (settings.py): /home/facetoface/makers/data
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - PDFs pour ce notebook seront stockés dans : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/pdfs
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Métadonnées pour ce notebook seront stockées dans : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/metadata
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - --- Configuration d'Embedding Active (depuis settings.py & .env) ---
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Fournisseur d'embedding par défaut configuré : ollama
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO -   Modèle Ollama Embedding : nomic-embed-text (Dimension: 768)

### Étape 1: Téléchargement d'Articles depuis ArXiv

Cette section utilise les fonctions `search_arxiv_papers` et `execute_arxiv_downloads` (anciennement `download_papers`) du module `src.data_processing.arxiv_downloader`.

1.  **Recherche** : Interroge l'API ArXiv avec la `ARXIV_QUERY_NOTEBOOK` et les paramètres de tri/limite.
2.  **Téléchargement** : Pour chaque article trouvé, télécharge le fichier PDF et sauvegarde ses métadonnées dans un fichier JSON.

Les fichiers sont sauvegardés dans les répertoires `NB_PDF_OUTPUT_DIR` et `NB_METADATA_OUTPUT_DIR` définis précédemment.

In [2]:
logger.info(f"--- Début Étape 1: Téléchargement d'Articles ArXiv ---")
logger.info(f"Requête ArXiv : '{ARXIV_QUERY_NOTEBOOK}', Max résultats : {MAX_RESULTS_NOTEBOOK}")
logger.info(f"Répertoire de sortie PDF : {NB_PDF_OUTPUT_DIR}")
logger.info(f"Répertoire de sortie Metadata : {NB_METADATA_OUTPUT_DIR}")

# Initialiser une variable pour les statistiques de téléchargement
download_stats = {"successful": 0, "failed": 0, "skipped": 0, "total": 0}
search_results_list: List[Any] = [] # Pour stocker les résultats de recherche ArXiv

try:
    # Étape 1.1: Recherche des articles sur ArXiv
    logger.info(f"Recherche en cours sur ArXiv...")
    search_results_list = search_arxiv_papers(
        query=ARXIV_QUERY_NOTEBOOK,
        max_results=MAX_RESULTS_NOTEBOOK,
        sort_by=settings.ARXIV_SORT_BY, 
        sort_order=settings.ARXIV_SORT_ORDER 
    )
    
    if not search_results_list:
        logger.warning("Aucun article trouvé sur ArXiv pour cette requête. Vérifiez la requête ou les paramètres.")
    else:
        logger.info(f"{len(search_results_list)} article(s) trouvé(s) sur ArXiv. Début du téléchargement...")
        
        # Étape 1.2: Téléchargement des articles trouvés
        # La fonction execute_arxiv_downloads gère le délai et la journalisation interne.
        download_stats = execute_arxiv_downloads(
            results=search_results_list,
            pdf_dir=NB_PDF_OUTPUT_DIR,
            metadata_dir=NB_METADATA_OUTPUT_DIR
        )
        logger.info(f"Processus de téléchargement ArXiv terminé.")

except Exception as e:
    logger.error(f"Une erreur majeure est survenue lors de la recherche ou du téléchargement ArXiv : {e}", exc_info=True)
    # Mettre à jour les stats pour refléter l'échec si l'erreur s'est produite avant que download_stats ne soit peuplé
    if not search_results_list: # Si l'erreur est survenue pendant la recherche
        download_stats["total"] = MAX_RESULTS_NOTEBOOK # Estimation
        download_stats["failed"] = MAX_RESULTS_NOTEBOOK # Estimation
    elif download_stats["successful"] == 0 and download_stats["skipped"] == 0 : # Si erreur pendant le DL mais stats non màj
        download_stats["total"] = len(search_results_list)
        download_stats["failed"] = len(search_results_list)


# --- Bilan du Téléchargement ---
logger.info(f"--- Bilan du Téléchargement ArXiv ---")
logger.info(f"  Articles recherchés (max) : {MAX_RESULTS_NOTEBOOK}")
logger.info(f"  Articles trouvés par ArXiv : {len(search_results_list)}")
logger.info(f"  Articles traités pour téléchargement : {download_stats.get('total', len(search_results_list))}") # .get pour si stats pas peuplé
logger.info(f"  Téléchargements réussis : {download_stats.get('successful', 0)}")
logger.info(f"  Téléchargements échoués : {download_stats.get('failed', 0)}")
logger.info(f"  Téléchargements sautés (déjà existants) : {download_stats.get('skipped', 0)}")

# Lister les fichiers téléchargés pour vérification (utile dans un notebook)
if download_stats.get('successful', 0) > 0 or download_stats.get('skipped', 0) > 0 :
    downloaded_pdfs = list(NB_PDF_OUTPUT_DIR.glob("*.pdf"))
    downloaded_metadata_files = list(NB_METADATA_OUTPUT_DIR.glob("*.json"))
    logger.info(f"  Vérification: {len(downloaded_pdfs)} fichier(s) PDF dans {NB_PDF_OUTPUT_DIR}.")
    if downloaded_pdfs:
        logger.info(f"    Exemple PDF: {downloaded_pdfs[0].name}")
    logger.info(f"  Vérification: {len(downloaded_metadata_files)} fichier(s) JSON dans {NB_METADATA_OUTPUT_DIR}.")
    if downloaded_metadata_files:
        logger.info(f"    Exemple Metadata: {downloaded_metadata_files[0].name}")
else:
    logger.warning("Aucun nouvel article n'a été téléchargé avec succès. Les étapes suivantes pourraient ne pas avoir de données à traiter.")

logger.info("--- Fin Étape 1 ---")

2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - --- Début Étape 1: Téléchargement d'Articles ArXiv ---
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Requête ArXiv : 'What are the latest advancements in using large language models for robot task planning?', Max résultats : 3
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Répertoire de sortie PDF : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/pdfs
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Répertoire de sortie Metadata : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/metadata
2025-06-05 20:59:04 - nb_01_ingestion_embedding - INFO - Recherche en cours sur ArXiv...
2025-06-05 20:59:04 - src.data_processing.arxiv_downloader - INFO - Searching ArXiv with query: What are the latest advancements in using large language models for robot task planning?
2025-06-05 20:59:04 - arxiv - INFO - Requesting page (first: True, try

### Étape 2: Parsing des Documents Téléchargés

Après le téléchargement, cette étape cruciale consiste à :
1.  **Lire les fichiers PDF** pour en extraire le contenu textuel brut.
2.  **Charger les fichiers de métadonnées JSON** associés à chaque PDF.
3.  **Combiner** le texte extrait et les métadonnées dans une structure de données unifiée (une liste d'objets `ParsedDocument` ou de dictionnaires similaires) pour chaque article.

Nous utilisons la fonction `parse_document_collection` du module `src.data_processing.document_parser`. Cette fonction prend en argument les chemins vers les répertoires contenant les PDFs et les métadonnées.

In [3]:
logger.info(f"--- Début Étape 2: Parsing des Documents ---")
logger.info(f"Répertoire des PDFs à parser : {NB_PDF_OUTPUT_DIR}")
logger.info(f"Répertoire des métadonnées à parser : {NB_METADATA_OUTPUT_DIR}")

# Initialiser la liste des documents parsés
# parse_document_collection retourne List[ParsedDocument], où ParsedDocument est un TypedDict.
parsed_documents: List[ParsedDocument] = [] 

# Vérifier si des PDFs existent pour le parsing (suite à l'étape de téléchargement)
pdf_files_for_parsing = list(NB_PDF_OUTPUT_DIR.glob("*.pdf"))
metadata_files_for_parsing = list(NB_METADATA_OUTPUT_DIR.glob("*.json"))

if not pdf_files_for_parsing:
    logger.warning(f"Aucun fichier PDF trouvé dans {NB_PDF_OUTPUT_DIR}. L'étape de parsing sera sautée.")
elif not metadata_files_for_parsing:
    logger.warning(f"Aucun fichier de métadonnées JSON trouvé dans {NB_METADATA_OUTPUT_DIR}. Le parsing pourrait manquer d'informations contextuelles.")
    # On peut décider de continuer ou d'arrêter. Pour ce notebook, on continue en signalant.
    
if pdf_files_for_parsing: 
    logger.info(f"Début du parsing pour {len(pdf_files_for_parsing)} PDF(s) potentiels...")
    try:
        # Appel de la fonction de parsing
        parsed_documents = parse_document_collection(
            pdf_dir=NB_PDF_OUTPUT_DIR,
            metadata_dir=NB_METADATA_OUTPUT_DIR 
        )
        
        if parsed_documents:
            logger.info(f"{len(parsed_documents)} document(s) ont été parsé(s) avec succès.")
            # Afficher des informations sur le premier document parsé (si disponible)
            first_doc_parsed = parsed_documents[0]
            # CORRECTION: Utiliser l'accès par clé pour TypedDict
            logger.info(f"  Exemple de document parsé (ID ArXiv): {first_doc_parsed.get('arxiv_id', 'N/A')}")
            logger.info(f"  Titre: {first_doc_parsed.get('metadata', {}).get('title', 'N/A')[:100]}...")
            logger.info(f"  Extrait du texte (100 premiers car.): {first_doc_parsed.get('text_content', '')[:100].replace(chr(10), ' ')}...")
            logger.info(f"  Nombre de pages: {first_doc_parsed.get('metadata', {}).get('page_count', 'N/A')}")
        else:
            logger.warning("Aucun document n'a pu être parsé. Vérifiez les logs précédents pour des erreurs spécifiques (ex: PDF corrompu).")
            
    except Exception as e:
        logger.error(f"Une erreur majeure est survenue lors du parsing des documents : {e}", exc_info=True)
        # S'assurer que parsed_documents est une liste vide en cas d'erreur
        parsed_documents = []

# Bilan du Parsing
if not parsed_documents:
    logger.warning("Bilan Parsing: Aucun document n'a été parsé avec succès. Les étapes suivantes (prétraitement, embedding, stockage) seront impactées ou sautées.")
else:
    logger.info(f"Bilan Parsing: {len(parsed_documents)} document(s) prêt(s) pour le prétraitement.")

logger.info("--- Fin Étape 2 ---")

2025-06-05 20:59:38 - nb_01_ingestion_embedding - INFO - --- Début Étape 2: Parsing des Documents ---
2025-06-05 20:59:38 - nb_01_ingestion_embedding - INFO - Répertoire des PDFs à parser : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/pdfs
2025-06-05 20:59:38 - nb_01_ingestion_embedding - INFO - Répertoire des métadonnées à parser : /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/metadata
2025-06-05 20:59:38 - nb_01_ingestion_embedding - INFO - Début du parsing pour 3 PDF(s) potentiels...
2025-06-05 20:59:38 - src.data_processing.document_parser - INFO - Found 3 PDF files in /home/facetoface/makers/data/corpus/what_are_the_latest_advancements_in_using_large_la/pdfs to parse
2025-06-05 20:59:38 - src.data_processing.document_parser - INFO - Parsing PDF: 2506.04228v1.pdf
2025-06-05 20:59:39 - src.data_processing.document_parser - INFO - Successfully processed and added document: 2506.04228
2025-06-05 20:59:3

### Étape 3: Prétraitement des Documents Parsés (Chunking)

Le texte brut extrait de chaque article est souvent volumineux. Pour le rendre exploitable par les modèles d'embedding et les LLMs (qui ont des limites de taille de contexte), nous devons le segmenter.

Cette étape, appelée "chunking", consiste à :
1.  Prendre le contenu textuel de chaque `ParsedDocument`.
2.  Le diviser en morceaux (chunks) plus petits, tout en essayant de préserver la cohérence sémantique (par exemple, en ne coupant pas au milieu d'une phrase si possible, ou en utilisant un chevauchement entre les chunks).
3.  Associer à chaque chunk les métadonnées pertinentes du document d'origine et des informations spécifiques au chunk (comme son numéro de séquence).

Nous utilisons `preprocess_parsed_documents` de `src.data_processing.preprocessor`. Les paramètres de chunking (taille `CHUNK_SIZE`, chevauchement `CHUNK_OVERLAP`) sont définis dans `config/settings.py`.

In [4]:
logger.info(f"--- Début Étape 3: Prétraitement des Documents (Chunking) ---")

# Initialiser la liste des chunks
chunks_with_metadata: List[Dict[str, Any]] = [] # La fonction retourne une liste de dictionnaires

if not parsed_documents:
    logger.warning("Aucun document parsé disponible pour le prétraitement (chunking). Étape sautée.")
else:
    logger.info(f"Début du prétraitement pour {len(parsed_documents)} document(s) parsé(s).")
    logger.info(f"Paramètres de chunking (depuis settings.py): CHUNK_SIZE={settings.CHUNK_SIZE}, CHUNK_OVERLAP={settings.CHUNK_OVERLAP}")
    
    try:
        # La fonction preprocess_parsed_documents prend List[ParsedDocument]
        # et retourne une List[Dict[str, Any]], où chaque dict est un chunk.
        chunks_with_metadata = preprocess_parsed_documents(parsed_documents)
        
        if chunks_with_metadata:
            logger.info(f"{len(chunks_with_metadata)} chunks ont été générés au total.")
            # Afficher des informations sur le premier chunk (si disponible) pour vérification
            first_chunk_example = chunks_with_metadata[0]
            logger.info(f"  Exemple de chunk généré (ID ArXiv d'origine): {first_chunk_example.get('metadata', {}).get('arxiv_id', 'N/A')}")
            logger.info(f"    Titre d'origine: {first_chunk_example.get('metadata', {}).get('original_document_title', 'N/A')[:70]}...")
            logger.info(f"    Numéro du chunk: {first_chunk_example.get('metadata', {}).get('chunk_sequence_number', 'N/A')}")
            logger.info(f"    Taille du texte du chunk: {len(first_chunk_example.get('text_chunk', ''))} caractères")
            logger.info(f"    Extrait du texte du chunk (100 premiers car.): {first_chunk_example.get('text_chunk', '')[:100].replace(chr(10), ' ')}...")
        else:
            logger.warning("Aucun chunk n'a été généré. Vérifiez le contenu des documents (pourraient être trop courts) ou les paramètres de chunking.")
            
    except Exception as e:
        logger.error(f"Une erreur majeure est survenue lors du prétraitement (chunking) des documents : {e}", exc_info=True)
        chunks_with_metadata = [] # S'assurer que la variable est une liste vide en cas d'erreur

# Bilan du Prétraitement
if not chunks_with_metadata:
    logger.warning("Bilan Prétraitement: Aucun chunk n'a été généré. Les étapes suivantes (embedding, stockage) seront impactées ou sautées.")
else:
    logger.info(f"Bilan Prétraitement: {len(chunks_with_metadata)} chunks prêts pour la génération d'embeddings.")

logger.info(f"--- Fin Étape 3 ---")

2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - --- Début Étape 3: Prétraitement des Documents (Chunking) ---
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - Début du prétraitement pour 3 document(s) parsé(s).
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - Paramètres de chunking (depuis settings.py): CHUNK_SIZE=1000, CHUNK_OVERLAP=200
2025-06-05 20:59:39 - src.data_processing.preprocessor - INFO - Starting preprocessing for 3 parsed documents.
2025-06-05 20:59:39 - src.data_processing.preprocessor - INFO - Preprocessing document 1/3: 2506.04228
2025-06-05 20:59:39 - src.data_processing.preprocessor - INFO - Preprocessing document 2/3: 2506.04227
2025-06-05 20:59:39 - src.data_processing.preprocessor - INFO - Preprocessing document 3/3: 2506.04229
2025-06-05 20:59:39 - src.data_processing.preprocessor - INFO - Finished preprocessing. Generated 47 chunks in total.
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - 47 chunks ont été générés au total.
2025

### Étape 4: Génération des Embeddings pour les Chunks

Les chunks de texte doivent être transformés en représentations vectorielles numériques (embeddings) pour permettre des recherches de similarité sémantique. Chaque chunk sera associé à un vecteur d'embedding.

Le fournisseur d'embedding (Ollama, OpenAI, HuggingFace Sentence Transformers, etc.) et le modèle spécifique sont déterminés par les configurations dans `config/settings.py` (et chargées via `.env`). Ce notebook utilisera la configuration active.

La fonction `generate_embeddings_for_chunks` du module `src.data_processing.embedder` est utilisée. Elle prend la liste des chunks (dictionnaires) et y ajoute une clé `embedding` contenant le vecteur.

In [5]:
logger.info(f"--- Début Étape 4: Génération des Embeddings ---")

# Initialiser la liste des chunks avec embeddings
# La fonction modifie les dictionnaires en place et les retourne.
embedded_chunks: List[Dict[str, Any]] = [] 

if not chunks_with_metadata:
    logger.warning("Aucun chunk disponible pour la génération d'embeddings. Étape sautée.")
else:
    logger.info(f"Début de la génération des embeddings pour {len(chunks_with_metadata)} chunk(s).")
    logger.info(f"Utilisation du fournisseur d'embedding configuré: {settings.DEFAULT_EMBEDDING_PROVIDER.upper()}")
    
    # Rappel des configurations spécifiques au fournisseur (déjà affiché mais utile ici)
    if settings.DEFAULT_EMBEDDING_PROVIDER == 'ollama':
        logger.info(f"  Modèle Ollama: {settings.OLLAMA_EMBEDDING_MODEL_NAME}, URL: {settings.OLLAMA_BASE_URL}")
        logger.info(f"  Assurez-vous que le serveur Ollama est actif et le modèle '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est accessible (ex: 'ollama pull {settings.OLLAMA_EMBEDDING_MODEL_NAME}').")
    elif settings.DEFAULT_EMBEDDING_PROVIDER == 'openai':
        logger.info(f"  Modèle OpenAI: {settings.OPENAI_EMBEDDING_MODEL_NAME}")
        if not settings.OPENAI_API_KEY:
             logger.error("  ATTENTION: OPENAI_API_KEY n'est pas configurée. L'embedding via OpenAI échouera.")
    # Ajouter d'autres fournisseurs si nécessaire

    try:
        # La fonction generate_embeddings_for_chunks prend List[Dict[str, Any]] (chunks)
        # et retourne la même liste, chaque dictionnaire étant enrichi d'une clé 'embedding'.
        embedded_chunks = generate_embeddings_for_chunks(chunks_with_metadata) # Passe une copie pour éviter modif en place si on réexécute la cellule
        
        if embedded_chunks and 'embedding' in embedded_chunks[0] and embedded_chunks[0]['embedding'] is not None:
            logger.info(f"Embeddings générés avec succès pour {len(embedded_chunks)} chunk(s).")
            first_embedded_chunk = embedded_chunks[0]
            embedding_sample = first_embedded_chunk['embedding']
            logger.info(f"  Exemple d'embedding pour le premier chunk (ID ArXiv d'origine: {first_embedded_chunk.get('metadata', {}).get('arxiv_id', 'N/A')}):")
            logger.info(f"    Type de l'embedding: {type(embedding_sample)}")
            logger.info(f"    Dimension de l'embedding: {len(embedding_sample) if isinstance(embedding_sample, list) else 'N/A'}")
            logger.info(f"    Extrait de l'embedding (5 premières valeurs): {embedding_sample[:5]}...")
            
            # Vérification de la dimension attendue
            expected_dim = 0
            if settings.DEFAULT_EMBEDDING_PROVIDER == 'ollama':
                expected_dim = settings.OLLAMA_EMBEDDING_MODEL_DIMENSION
            elif settings.DEFAULT_EMBEDDING_PROVIDER == 'openai':
                expected_dim = settings.OPENAI_EMBEDDING_DIMENSION
            elif settings.DEFAULT_EMBEDDING_PROVIDER == 'huggingface':
                expected_dim = settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION
            
            if expected_dim > 0 and isinstance(embedding_sample, list) and len(embedding_sample) != expected_dim:
                logger.warning(f"  ATTENTION: La dimension de l'embedding généré ({len(embedding_sample)}) ne correspond pas à la dimension attendue ({expected_dim}) configurée dans settings.py pour {settings.DEFAULT_EMBEDDING_PROVIDER}.")
            elif expected_dim > 0:
                 logger.info(f"    La dimension de l'embedding ({len(embedding_sample)}) correspond à la dimension attendue ({expected_dim}).")

        elif embedded_chunks and ('embedding' not in embedded_chunks[0] or embedded_chunks[0]['embedding'] is None):
            logger.warning(f"La génération d'embeddings semble avoir retourné des données, mais la clé 'embedding' est manquante ou vide pour le premier chunk. Vérifiez le processus d'embedding.")
        else:
            logger.warning("Aucun embedding n'a été généré ou retourné. Vérifiez les logs du fournisseur d'embedding.")
            
    except Exception as e:
        logger.error(f"Une erreur majeure est survenue lors de la génération des embeddings : {e}", exc_info=True)
        embedded_chunks = [] # S'assurer que la variable est une liste vide en cas d'erreur

# Bilan de la Génération d'Embeddings
if not embedded_chunks or not ('embedding' in embedded_chunks[0] and embedded_chunks[0]['embedding'] is not None):
    logger.warning("Bilan Embeddings: Aucun embedding valide n'a été généré. Le stockage dans MongoDB sera impacté ou sauté.")
else:
    logger.info(f"Bilan Embeddings: {len(embedded_chunks)} chunks avec embeddings sont prêts pour le stockage.")

logger.info(f"--- Fin Étape 4 ---")

2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - --- Début Étape 4: Génération des Embeddings ---
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - Début de la génération des embeddings pour 47 chunk(s).
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO - Utilisation du fournisseur d'embedding configuré: OLLAMA
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO -   Modèle Ollama: nomic-embed-text, URL: http://localhost:11434
2025-06-05 20:59:39 - nb_01_ingestion_embedding - INFO -   Assurez-vous que le serveur Ollama est actif et le modèle 'nomic-embed-text' est accessible (ex: 'ollama pull nomic-embed-text').
2025-06-05 20:59:39 - src.data_processing.embedder - INFO - Starting embedding generation for 47 chunks.
2025-06-05 20:59:39 - src.data_processing.embedder - INFO - Initializing embedding client for provider: ollama
2025-06-05 20:59:39 - src.data_processing.embedder - INFO - Using OllamaEmbeddings with model: nomic-embed-text via http://localhost:11434
202

/home/facetoface/makers/src/data_processing/embedder.py:75: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  return OllamaEmbeddings(


2025-06-05 20:59:40 - src.data_processing.embedder - INFO - Embedding batch 2/2 (size: 15) using ollama provider.
2025-06-05 20:59:41 - src.data_processing.embedder - INFO - Finished embedding generation. Successfully structured 47 chunks for DB out of 47.
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - Embeddings générés avec succès pour 47 chunk(s).
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -   Exemple d'embedding pour le premier chunk (ID ArXiv d'origine: 2506.04228):
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -     Type de l'embedding: <class 'list'>
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -     Dimension de l'embedding: 768
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -     Extrait de l'embedding (5 premières valeurs): [1.0089068412780762, 0.9489849805831909, -2.6395182609558105, -0.37363749742507935, 0.31627190113067627]...
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -     La dimension de l'embedding (768) corr

### Étape 5: Stockage des Chunks et Embeddings dans MongoDB

Les chunks de texte, enrichis de leurs métadonnées et de leurs embeddings vectoriels, sont maintenant prêts à être stockés dans une base de données vectorielle. Nous utilisons MongoDB Atlas comme backend, qui supporte la recherche vectorielle.

Cette étape implique :
1.  **Connexion à MongoDB** : Établir une connexion à l'instance MongoDB spécifiée dans `MONGODB_URI`.
2.  **Préparation des Données** : S'assurer que les données sont dans un format adéquat pour l'insertion.
3.  **Insertion des Données** : Insérer les chunks (avec texte, métadonnées, et embedding) dans la collection MongoDB spécifiée (`COLLECTION_NAME_NOTEBOOK`).
4.  **Création d'Index** :
    *   Créer un **index de recherche vectorielle** sur le champ `embedding` pour permettre des recherches de similarité rapides.
    *   Optionnellement, créer des **index textuels** sur d'autres champs (comme `text_chunk` ou des champs de métadonnées) pour des recherches hybrides ou par mots-clés.

Nous utilisons la classe `MongoDBManager` de `src.vector_store.mongodb_manager` pour gérer ces opérations.

In [6]:
logger.info(f"--- Début Étape 5: Stockage dans MongoDB ---")

if not embedded_chunks or not ('embedding' in embedded_chunks[0] and embedded_chunks[0]['embedding'] is not None):
    logger.warning("Aucun chunk avec embedding valide disponible pour le stockage dans MongoDB. Étape sautée.")
else:
    logger.info(f"Préparation au stockage de {len(embedded_chunks)} chunks dans MongoDB.")
    logger.info(f"  Base de données cible : {settings.MONGO_DATABASE_NAME}")
    logger.info(f"  Collection cible pour ce notebook : {COLLECTION_NAME_NOTEBOOK}")

    mongo_mgr = None 
    try:
        logger.info("Initialisation du MongoDBManager...")
        mongo_mgr = MongoDBManager(
            mongo_uri=settings.MONGODB_URI,
            db_name=settings.MONGO_DATABASE_NAME
        )
        mongo_mgr.connect() 
        logger.info(f"Connecté à MongoDB (Base de données: {mongo_mgr.db.name}).")

        CLEAN_COLLECTION_BEFORE_INSERT = True 
        if CLEAN_COLLECTION_BEFORE_INSERT:
            if mongo_mgr.collection_exists(COLLECTION_NAME_NOTEBOOK): 
                logger.warning(f"La collection '{COLLECTION_NAME_NOTEBOOK}' existe déjà. Tentative de suppression pour ce test de notebook...")
                mongo_mgr.db[COLLECTION_NAME_NOTEBOOK].drop()
                logger.info(f"Collection '{COLLECTION_NAME_NOTEBOOK}' supprimée.")
            else:
                logger.info(f"La collection '{COLLECTION_NAME_NOTEBOOK}' n'existe pas encore, aucune suppression nécessaire.")
        
        logger.info(f"Insertion de {len(embedded_chunks)} chunks dans la collection '{COLLECTION_NAME_NOTEBOOK}'...")
        insert_summary = mongo_mgr.insert_chunks_with_embeddings(
            chunks=embedded_chunks, 
            collection_name=COLLECTION_NAME_NOTEBOOK
        )
        logger.info(f"Résumé de l'insertion : {insert_summary}")

        if insert_summary.get("inserted_count", 0) > 0:
            expected_dimension_for_log = 0
            try:
                expected_dimension_for_log = mongo_mgr.get_effective_embedding_dimension()
            except ValueError:
                logger.warning("Impossible de déterminer la dimension d'embedding attendue depuis les settings.")

            logger.info(f"Création de l'index de recherche vectorielle '{VECTOR_INDEX_NAME_NOTEBOOK}' sur le champ 'embedding'. Dimension attendue: {expected_dimension_for_log if expected_dimension_for_log > 0 else 'auto-détectée par le manager'}.")
            
            vector_index_filter_fields = [
                "metadata.arxiv_id",
                "metadata.original_document_title",
                "metadata.primary_category",
                "metadata.published_year" 
            ]
            # CORRECTION 2: Retirer également 'similarity_metric' de l'appel
            mongo_mgr.create_vector_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=VECTOR_INDEX_NAME_NOTEBOOK,
                embedding_field="embedding", 
                filter_fields=vector_index_filter_fields 
            )
            logger.info(f"Requête de création de l'index vectoriel '{VECTOR_INDEX_NAME_NOTEBOOK}' soumise. Cela peut prendre quelques minutes pour être actif sur Atlas.")

            logger.info(f"Création de l'index textuel '{TEXT_INDEX_NAME_NOTEBOOK}' sur le champ 'text_chunk' et certains champs de métadonnées...")
            text_index_additional_fields = {
                "metadata.original_document_title": "text", 
                "metadata.summary": "text" 
            }
            mongo_mgr.create_text_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=TEXT_INDEX_NAME_NOTEBOOK,
                text_field="text_chunk", 
                additional_text_fields=text_index_additional_fields
            )
            logger.info(f"Requête de création de l'index textuel '{TEXT_INDEX_NAME_NOTEBOOK}' soumise.")
            
            logger.info("Stockage dans MongoDB et création des index terminés.")
        else:
            logger.warning("Aucun document n'a été inséré. La création des index sera sautée.")

    except ConnectionFailure:
        logger.error("ERREUR CRITIQUE : Échec de la connexion à MongoDB. Vérifiez votre MONGODB_URI et l'accessibilité du serveur.", exc_info=True)
    except Exception as e:
        logger.error(f"Une erreur majeure est survenue lors de l'interaction avec MongoDB : {e}", exc_info=True)
    finally:
        if mongo_mgr:
            logger.info("Fermeture de la connexion MongoDB.")
            mongo_mgr.close()

logger.info(f"--- Fin Étape 5 ---")

2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - --- Début Étape 5: Stockage dans MongoDB ---
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - Préparation au stockage de 47 chunks dans MongoDB.
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -   Base de données cible : makers_db
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO -   Collection cible pour ce notebook : arxiv_chunks_nb_test_ollama
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - Initialisation du MongoDBManager...
2025-06-05 20:59:41 - src.vector_store.mongodb_manager - INFO - Initialized MongoDB manager for database: makers_db
2025-06-05 20:59:41 - src.vector_store.mongodb_manager - INFO - Connected to MongoDB database: makers_db
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - Connecté à MongoDB (Base de données: makers_db).
2025-06-05 20:59:41 - nb_01_ingestion_embedding - INFO - La collection 'arxiv_chunks_nb_test_ollama' n'existe pas encore, aucune suppression nécessaire.
2

## Fin du Notebook 01: Ingestion et Embedding

Ce notebook a couvert l'ensemble du pipeline d'ingestion de données ArXiv :
- Téléchargement des articles et de leurs métadonnées.
- Parsing des PDFs pour extraire le texte.
- Prétraitement du texte par segmentation (chunking).
- Génération des embeddings vectoriels pour chaque chunk.
- Stockage des chunks enrichis dans MongoDB Atlas, avec création d'index vectoriels et textuels.

**Prochaines Étapes Suggérées :**
- **Explorer les Données Stockées** : Vous pouvez utiliser MongoDB Compass ou un autre notebook pour interroger la collection `COLLECTION_NAME_NOTEBOOK` et examiner les documents stockés.
- **Notebook 02: Exploration des Stratégies RAG** : Utiliser les données ingérées pour tester différentes approches de recherche sémantique et de génération de réponses.

Si des erreurs sont survenues, veuillez examiner attentivement les logs de chaque étape pour diagnostiquer le problème. Assurez-vous que toutes les configurations (fichier `.env`, serveur Ollama, accès MongoDB) sont correctes.